## test find

In [ ]:
# test find(e,element_list)

import random
# ==========================data structure==========================
class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'count: {}'.format(self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self,count=1):
        super().__init__(count)
        self.distinct = hyperloglog.HyperLogLog(0.01)
        self.maxID=''
    def __str__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))
    def __repr__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
    def __repr__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
# ==========================data structure==========================

def find(e,element_list):
    try:
        index = [ele.ID for ele in element_list].index(e.ID)
    except:
        index=-99
    return index

filename='kosarak_ground_truth.csv'
filepath="..\\result\\kosarak\\"
src_data=os.path.join(filepath,filename)
df=pd.read_csv(src_data)
x=[Tail(i,j) for i,j in df.values.tolist()]
y=[]
randomindex=[random.randint(1,len(x)) for i in range(100)]
for index in randomindex:
    y.append(x[index])
for item in y:
    result=find(item,x)
    print("item in y: {},find index:{},item in x[{}]:{}".format(item,result,result,x[result]))
    print("item?=x[result]:{}".format(item==x[result]))

## UpdateSK

In [50]:
import ds
import spookyhash
import mmh3
from numpy import random
import os
import time
import operator
import hyperloglog
from tqdm import tqdm

# ==========================data structure==========================
class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'count: {}'.format(self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self,count=1):
        super().__init__(count)
        self.distinct = hyperloglog.HyperLogLog(0.01)
        self.maxID=''
        
    def __str__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))
    def __repr__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
    def __repr__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
# ==========================data structure==========================



# ========================== Update Sketch==========================
def UpdateSk(element,width,depth):
    e_max=get_emax()
    hash1=spookyhash.hash32(bytes((element.ID),encoding='utf-8'))
        # input of spooky hash: binary, encoding is parameter of bytes()
        # output of spooky hash: unsigned- 32 bit int
    hash2=mmh3.hash(str(hash1), signed=False)
        # input of mmh3 is str, output unsigned- 32 bit int
    row=hash1 % depth
        # index of row in SK[row]
    ID=hash2 % ((width*3)//2)
        # hash-value(ID) of e
        # range: 2^k *3/2= 3*(2^k-1)
    match=False
        # match= True if e is in Sk[row]
    index=0
    avg=0
    item=Tail(ID, element.count)
        # ID in Sketch is hash value
    print("\t{} -> {},send to Sk[{}]".format(element,item,row))
    # print("id(Sk[{}]):{}".format(row,id(Sk[row])))
    if len(Sk[row])==0:
        # Sk[row] is empty,append e directly
        Sk[row].append(item)
        Sk_head[row].maxID=element.ID
        match=True
        index=0
    else:
        # Sk[row] is not empty
        index=find(item,Sk[row])
        # print("index={}".format(index))
        if index >0:
            # matches in Sk[row]
            Sk[row][index].count+=item.count
            match=True
        else:
            # doesn't match in Sk[row]
            if len(Sk[row])<width:
                Sk[row].append(item)
                match=True
                index=len(Sk[row])-1
            else:
                # Sk[row] is full,
                count_sum=sum(Sk[row][i].count for i in range(len(Sk[row])))
                # print("count_sum={}".format(count_sum))
                # print("total count={}".format(Sk[row][0].count))
                if Sk_head[row].count==count_sum:
                    # no element in other
                    # print("no element in Other, send to distinct")
                    Sk_head[row].distinct.add(element.ID)
                    match=False
                elif Sk_head[row].count>count_sum:
                    avg=(Sk_head[row].count-count_sum)//len(Sk_head[row].distinct)
                    if avg>Sk[row][-1].count:
                        # print("avg>last one, update Sk[{}]".format(row))
                        Sk[row][-1].ID=item.ID
                        Sk[row][-1].count=avg
                        match=True
                        index=-1
                    else:
                        Sk_head[row].distinct.add(element.ID)
                        # print("avg< last one,send to distinct")
                        match=False
    Sk_head[row].count+=item.count
        # total count+=count
    # Sk[row].sort(key=operator.attrgetter('count'),reverse=True)
#============================start here==========================================================
# need to memory max element each row, update e_max with max element.
#============================start here==========================================================

    # update e_max in Sketch
    if match:
        # e.count in Sk[row]> e_max.count
        if Sk[row][index].count>e_max.count:
            e_max.ID=element.ID
            e_max.count=Sk[row][index].count
    else:
        # avg(e.count) in other > e_max.count
        if avg >e_max.count:
            e_max=Tail(element.ID,avg)
    Sk[row].sort(key=operator.attrgetter('count'),reverse=True)
    
    # print("after sorting id(Sk[{}]):{}".format(row,id(Sk[row])))

    for i in range(len(Sk)):
        print("Sk[{}]:{},{}".format(i,Sk_head[i],Sk[i]))
    print("e_max:{} after UpdateSK()".format(e_max))

# ========================== Update Sketch==========================
# ========================== BringBack=========================
def BringBack(e_min,e_max):
    print("Top before Bringback:\n\t{}".format(Top))
    e_max=get_emax()
    # print('e_max at first Bringback:{},id(e_max):{}'.format(e_max,id(e_max)))
    temp=Tail(e_min.ID,e_min.count)
    Top[-1].ID=e_max.ID
    Top[-1].count=e_max.count
    Top.sort(key=operator.attrgetter('count'),reverse=True)
    print('Top after BringBack:\n\t{}'.format(Top))
    DeleteSk(e_max)
        # e_max in Sk[row]
    e_max.ID=''
    e_max.count=0
    print("e_max after delete:{},id(e_max):{}".format(e_max,id(e_max)))
    UpdateSk(temp,width,depth)
    
    # print("Sk[] after Update {}:\n\t{}".format(e_min,Sk))

# ========================== BringBack=========================
# ==========================DeleteSk=========================
def DeleteSk(element):
    # 刪除e_max in Sk[row]，找新的e_max
    hash1=spookyhash.hash32(bytes((element.ID),encoding='utf-8'))
    hash2=mmh3.hash(str(hash1), signed=False)
    ID=hash2 % ((width*3)//2)
    row=hash1 % depth
    Sk_head[row].count-=element.count
    index=find(ID,Sk[row])
    if index<0:
        pass
    else:
        Sk[row].pop(index)
    '''
    print("index:{}".format(index))
    print('After pop:')
    print("\tSk[{}]={}".format(row,Sk[row]))
    print("Top:\n\t{}".format(Top))
    '''
# ==========================DeleteSk=========================
def get_emax():
    return e_max

def find(e,element_list):
    try:
        # type(element_list[i].ID):int
        # print("e:{}".format(e))
        # print("element_list[1:]: {}".format(element_list[1:]))
        index=[ele.ID for ele in element_list].index(e.ID)
    except:
        # print("raise error")
        index=-99
    return index

# ==================================================================

start=time.time()
filename='kosarak.dat'
filepath="..\\dataset\\"
src_data=os.path.join(filepath,filename)
depth=4
width=8
size=16

Top=[]
Sk_head=[Head(0) for j in range(depth)]
Sk=[[] for i in range(depth)]
item_count=100
e_max=Tail('',0)

with open(src_data,'r') as file:
    while item_count:
        element=file.readline().strip('\n')
        if not element:
            break
        else:
            print("read {}-th element: {}".format(-(item_count-101),element))
            item_count-=1
            item=Tail(element,1)
            if len(Top)==0:
                Top.append(item)
                # print("len(Top)==0, Top append:{}".format(Top))
            else:
                index=find(item,Top)
                # print("index={}".format(index))
                if index<0:
                    if len(Top)<size:
                        Top.append(item)
                        # print("index={},Top after append: {},\nlen(Top):{}".format(index,Top,len(Top)))
                    else:
                        print("send {} into Sk".format(element))
                        UpdateSk(item,width,depth)
                else:
                    # print("update Top[{}]:".format(index))
                    Top[index].count+=1
                    print(Top)
            Top.sort(key=operator.attrgetter('count'),reverse=True)
            if e_max.count>Top[-1].count:
                BringBack(Top[-1],e_max)
                # print('Top after BringBack: \n\t{}'.format(Top))            
            # print("TOP:{}".format(Top))
end=time.time()
print("e_max:{}".format(e_max))
print("Execution time:{} seconds.".format(str(end-start)))

read 1-th element: 1 2 3
read 2-th element: 1
read 3-th element: 4 5 6 7
read 4-th element: 1 8
read 5-th element: 9 10
read 6-th element: 11 6 12 13 14 15 16
read 7-th element: 1 3 7
read 8-th element: 17 18
read 9-th element: 11 6 19 20 21 22 23 24
read 10-th element: 1 25 3
read 11-th element: 26 3
read 12-th element: 11 27 6 3 28 7 29 30 31 32 33 34 35 36 37
read 13-th element: 6 2 38
read 14-th element: 39 11 27 1 40 6 41 42 43 44 45 46 47 3 48 7 49 50 51
read 15-th element: 52 6 3 53
read 16-th element: 54 1 6 55
read 17-th element: 11 6 56 57 58 59 60 61 62 63 64
send 11 6 56 57 58 59 60 61 62 63 64 into Sk
	[ID: 11 6 56 57 58 59 60 61 62 63 64, count: 1] -> [ID: 4, count: 1],send to Sk[1]
Sk[0]:[total count: 0, distinct: 0],[]
Sk[1]:[total count: 1, distinct: 0],[[ID: 4, count: 1]]
Sk[2]:[total count: 0, distinct: 0],[]
Sk[3]:[total count: 0, distinct: 0],[]
e_max:[ID: 11 6 56 57 58 59 60 61 62 63 64, count: 1] after UpdateSK()
read 18-th element: 3
send 3 into Sk
	[ID: 3, coun

In [35]:
filename='kosarak.dat'
filepath="..\\dataset\\"
src_data=os.path.join(filepath,filename)


item_count=10000
x='3'
count=0
with open(src_data,'r') as file:
    while item_count:
        element=file.readline().strip('\n')
        if not element:
            break
        else:
            item_count-=1
            if element==x:
                count+=1
print(count)

381


## DeleteSK

In [ ]:


def DeleteSk(element):
    # Delete element in SK and update e_max
    hash1=spookyhash.hash32(bytes((element.ID),encoding='utf-8'))
    hash2=mmh3.hash(str(hash1), signed=False)
    ID=hash2 % ((width*3)//2)
    row=hash1 % depth
    Sk[row][0].count-=element.count
    index=find(ID,Sk[row])
    if index<0:
        pass
    else:
        Sk[row].pop(index)


filename='kosarak_ground_truth.csv'
filepath="..\\result\\kosarak\\"
src_data=os.path.join(filepath,filename)
df=pd.read_csv(src_data)
x=[Tail(i,j) for i,j in df.values.tolist()]

depth=4
width=128
Sk=[[ds.Head(0)] for j in range(depth)]


In [ ]:

import copy

x=copy.copy(Sk[3])
print(id(x),id(Sk[3]))
print(x)

def sort_Sk(element_list):
    tail=element_list[1:]
    tail.sort(key=operator.attrgetter('count'),reverse=True)
    return [element_list[0]]+tail
    
Sk[3]=sort_Sk(x)
print(Sk[3])

In [ ]:
a=[[] for i in range(3)]
a
a[1].append(3)
len(a)

In [3]:
import hyperloglog
class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'count: {}'.format(self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self,count=0):
        super().__init__(count)
        self.distinct = hyperloglog.HyperLogLog(0.01)
        # self.maxID=''
    def __str__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))
    def __repr__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
    def __repr__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
x=[[Head(0)] for j in range(10)]


[[[total count: 0, distinct: 0]],
 [[total count: 0, distinct: 0]],
 [[total count: 0, distinct: 0]],
 [[total count: 0, distinct: 0]],
 [[total count: 0, distinct: 0]],
 [[total count: 0, distinct: 0]],
 [[total count: 0, distinct: 0]],
 [[total count: 0, distinct: 0]],
 [[total count: 0, distinct: 0]],
 [[total count: 0, distinct: 0]]]

In [6]:
y=Tail('a',1)
y.count+=1
y

[ID: a, count: 2]

In [14]:
x=[Head(0) for j in range(4)]
x

[[total count: 0, distinct: 0],
 [total count: 0, distinct: 0],
 [total count: 0, distinct: 0],
 [total count: 0, distinct: 0]]